<a href="https://colab.research.google.com/github/Bryan-Az/Mathematics-LLM/blob/main/%5BTraining%5D_Mathematics_Model_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training the 'Storybook' Story Generation Model on a Distributed Environment
This notebook is running on the TPUv2-8 GPU environment in Kaggle. The pre-trained foundation model we are using is the gated meta-llama/Meta-Llama-3-8B-Instruct, requiring authentication with HuggingFace.

## Imports and Installs

In [ ]:
!pip install datasets
!pip3 install transformers zstandard jsonlines peft wandb bitsandbytes -q
!pip3 install accelerate datasets sentencepiece langchain torch_xla[tpuvm] -q
!pip uninstall -y tensorflow
!pip install tensorflow-cpu -q
!git clone https://github.com/IsNoobgrammer/Pytorch-Optimizers optims

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[

In [ ]:
import os
import pandas as pd
import numpy as np
import datasets
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch
import torch.nn as nn
import torch_xla.test.test_utils as test_utils
import torch_xla.experimental.xla_sharding as xs
import torch_xla.core.xla_model as xm
from transformers import (
 AutoTokenizer, AutoModelForCausalLM, set_seed, DataCollatorWithPadding, AutoConfig
)

from transformers import logging as hf_logging
import torch_xla.runtime as xr

xr.use_spmd()

from torch_xla.experimental.xla_sharding import Mesh

from peft import LoraConfig, TaskType, get_peft_model
from datasets import  load_dataset, concatenate_datasets
from tqdm import tqdm

from torch.utils.data import Dataset as TorchDataset
from torch_xla.utils.checkpoint import checkpoint

try:
    !export USE_TORCH=True #If we don't do this, transformers will seemingly bork the session upon import. Really weird error.
    os.environ["PJRT_DEVICE"] = "TPU"
    os.environ.pop('TPU_PROCESS_ADDRESSES')
    os.environ.pop('CLOUD_TPU_TASK_ID')
    hf_logging.set_verbosity_error() # It can still display warnings which is a bit annoying but whatever
except:
    pass

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_13/3511949292.py:10: DeprecationWarning: Importing from `torch_xla.experimental.xla_sharding` will be deprecated after 2.2 release. Please use `torch_xla.distributed.spmd` instead.
  import torch_xla.experimental.xla_sharding as xs


In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
# login with huggginface for using gated LLaMA foundational models
!huggingface-cli login --token $hf_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Loading the Tokenizer of the Pre-trained LlaMA 8B Model
It's necessary to import the tokenizer of the model for loading the dataset.

In [ ]:
MAX_INPUT=4096 #128*32
MODEL = "meta-llama/Meta-Llama-3-8B-Instruct" #You should be able to use 7B model with no changes! There should be enough HBM
SAVED_MODEL = "Alexis-Az/Story-Generation-LlaMA-8B"
# !export XLA_TENSOR_ALLOCATOR_MAXSIZE=1000000

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)
if 'pad_token' not in tokenizer.special_tokens_map:
  tokenizer.pad_token=tokenizer.eos_token


print(f"Tokens :\n {tokenizer.special_tokens_map} \n\n")

Tokens :
 {'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>', 'pad_token': '<|eot_id|>'} 




## Loading the Dataset

In [ ]:
#%%writefile spmd_util.py
import math
from dataclasses import dataclass, field
from typing import List, Optional
from collections import defaultdict
import torch
import torch.nn as nn
import re
import torch_xla.experimental.xla_sharding as xs
import torch_xla.core.xla_model as xm
from transformers import LlamaConfig

# these are model architecture formatting rules specific to LLaMA
LLAMA_RULES = (
    ("model\\.embed_tokens", ("mp", "fsdp")),
    ("self_attn\\.(q_proj|k_proj|v_proj)", ("fsdp", "mp")),
    ("self_attn\\.o_proj", ("mp", "fsdp")),
    ("mlp\\.gate_proj", ("fsdp", "mp")),
    ("mlp\\.down_proj", ("mp", "fsdp")),
    ("mlp\\.up_proj", ("fsdp", "mp")),
    ("lm_head", ("fsdp", "mp")),
)

strkey2id = {
    "dp": 0, ## usefull for sharding inputs
    "fsdp": 1, ## Pytorch-Xla (2D-sharding) axis to shard data (mostly mesh shape will be (8,1)) data will be sharded 8 way
    "mp": 2 ## axis to shard model model will be sharded one way
               ## Recommened checking Pytorch-tpu/transfomers on github (xla-fork of transformers)
}

def partition_module(model, mesh, device=xm.xla_device(), verbose=False):
    partition_specs = LLAMA_RULES
    rule = [(k, tuple([strkey2id[x] for x in v])) for k, v in partition_specs]

    # print(rule)

    for name, module in model.named_modules():
        module.to(device)
#         print(name, module.__class__.__name__)
        if isinstance(module, (nn.Embedding, nn.Linear)):
            for rule_pattern, spec in rule:
                if re.findall(rule_pattern, name.lower())  : # and ("lora" not in name.lower()):
                    if verbose:
                        print("match", rule_pattern, name)

                    xs.mark_sharding(module.weight, mesh, spec)
                    break

In [ ]:
class InstructionDataset(TorchDataset):
    def __init__(self, tokenizer, max_length=1024, dataset=None):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        prompts = self.dataset[idx]
        text = ""
        for prompt in prompts:
            #print(prompt)
            data = prompts[prompt]
            if prompt == 'instruction':
                #print(data)
                text += f"<|im_start|>user\n{data['prompt:']}<|im_end|>\n"
                ## TO-DO: add prompting for these
                #text += f"\n{data['words']}\n"
                #text += f"\n{data['features']}\n"
            if prompt == 'summary':
                text += f"<|im_start|>assistant\n Here's a really short story: {data}<|im_end|>"
            if prompt == 'story':\
                text += f"<|im_start|>assistant\n Here's the full story:{data}<|im_end|>"





        input_ids = self.tokenizer(text, add_special_tokens=True, max_length=self.max_length, truncation=True, padding="max_length", return_attention_mask=True, return_tensors="pt")
        return {
            "input_ids": input_ids["input_ids"].squeeze(0),
            "labels": input_ids["input_ids"].squeeze(0),
            "attention_mask":input_ids["attention_mask"].squeeze(0),
        }

In [ ]:
train_dataset="/kaggle/input/tinystories"
test_dataset="/kaggle/input/tinystories"
# ~1/5 of the dataset is used for validation
train_data = load_dataset(train_dataset, split="train").shuffle(seed=69)
val = (load_dataset(test_dataset, split="train[:1000000]")).shuffle(seed=420)

Generating train split: 4967871 examples [04:41, 17670.52 examples/s]


In [ ]:
FLAGS = {'MAX_INPUT': MAX_INPUT,
         'LOGGING_STEPS': 1,
         'NUM_EPOCHS': 1,
         'PAUSE_STEPS':1000, # asks to exit training after x steps #todo checkpoints
         'MAX_STEPS': -1,#Ooverides num epochs
         'BATCH_SIZE': 2, #Making batch_size lower then 8 will result in slower training, but will allow for larger models\context. Fortunately, we have 128GBs. Setting higher batch_size doesn't seem to improve time.
          'LEN_TRAIN_DATA': len(train_data),
         'VAL_STEPS': 20,
         'VAL_BATCH': 5,
         'GRAD_ACCUMULATION_STEP':1,
         'MAX_GRAD_CLIP':1,
        'LEARNING_RATE':6e-5,
         'WARMUP_RATIO':0.01,
         'OPTIMIZER':'SM3', # default = 'adamw'  options->  ['adamw','SM3','came','adafactor','lion']
         'SCHEDULAR':'cosine', # default= 'cosine'     options:-> ['linear','cosine']
         'WEIGHT_DECAY':0.1,
         'TRAIN_DATASET':train_dataset,
         "TEST_DATASET":test_dataset,
         'WANDB':True,
        'PROJECT':'Storybook-Model',
        }

In [ ]:
train_data = InstructionDataset(tokenizer, dataset=train_data, max_length=FLAGS['MAX_INPUT'])
val = InstructionDataset(tokenizer, dataset=val)
train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_data, num_replicas=8, rank=xm.get_ordinal(), shuffle=True,drop_last=True)
training_loader = torch.utils.data.DataLoader(train_data, batch_size=FLAGS["BATCH_SIZE"], sampler=train_sampler)
val_sampler = torch.utils.data.distributed.DistributedSampler(
    val, num_replicas=8, rank=xm.get_ordinal(), shuffle=True,drop_last=True)
testing_loader = torch.utils.data.DataLoader(val, batch_size=FLAGS["BATCH_SIZE"], sampler=val_sampler)

print(f"Max Steps: {len(training_loader)}, Batch size: {8*FLAGS['BATCH_SIZE']}")
print(f"Val Size: {len(testing_loader)}, Batch Size: {8*FLAGS['BATCH_SIZE']}")
FLAGS['STEPS']=len(training_loader)
FLAGS['BATCH_DATA']=FLAGS['BATCH_SIZE']*8 ## 8 CORES ON TPU

Max Steps: 310492, Batch size: 16
Val Size: 62500, Batch Size: 16


## Loading the Pre-trained Model with LoRa Adapters
Adding LoRa will allow us to fine-tune the model on our story dataset.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL,torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.25it/s]


In [ ]:
ls=LoraConfig(
    r = 48, # Lora Rank ,I would prefer 8-32 for smaller models like 7b
    target_modules = ['q_proj', 'down_proj', 'up_proj', 'o_proj', 'v_proj', 'gate_proj', 'k_proj'],
    lora_alpha = 16, #weight_scaling
    lora_dropout = 0.05, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimize
    # modules_to_save = ["lm_head", "embed_tokens"] ## if you use new chat formats or embedding tokens
)
model = get_peft_model(model, ls)
model.print_trainable_parameters()

trainable params: 125,829,120 || all params: 8,156,090,368 || trainable%: 1.5428


In [ ]:
import torch_xla.distributed.parallel_loader as pl

device = xm.xla_device()
model = model.to(device)
from torch_xla.distributed.fsdp.utils import apply_xla_patch_to_nn_linear
model = apply_xla_patch_to_nn_linear(model, xs.xla_patched_nn_linear_forward)  #for patching linear layer to use einsum instead of matmul
num_devices = xr.global_runtime_device_count()
model_axis=1
data_axis=num_devices//model_axis
mesh_shape = (1,data_axis, model_axis )
device_ids = np.array(range(num_devices))
mesh = Mesh(device_ids, mesh_shape, ('dp','fsdp', 'mp'))
partition_module(model, mesh)
training_loader = pl.MpDeviceLoader(training_loader, device)
testing_loader = pl.MpDeviceLoader(testing_loader, device)
mesh_shape

(1, 8, 1)

## Training the Model

In [ ]:
!export XLA_USE_BF16=1
import torch.nn as nn
import wandb
__wandb__=FLAGS['WANDB']
from torch_xla.amp.syncfree import AdamW
from transformers import get_linear_schedule_with_warmup,get_cosine_schedule_with_warmup
from optims.optim import SM3, CAME , Adafactor
# from random import randrange
# from bitsandbytes.optim import AdamW8bit
# from torchdistx.optimizers import AnyPrecisionAdamW

val_step=0




def evaluate_loss(outputs,labels,pad_id=tokenizer.pad_token_id):
  epsilon=1e-8
  logits=outputs.logits
  logits = logits[..., :-1, :].contiguous()
  labels = labels[..., 1:].contiguous()
  log_probs = -nn.functional.log_softmax(logits, dim=-1)
  if labels.dim() == log_probs.dim() - 1:
    labels = labels.unsqueeze(-1)
  padding_mask = labels.eq(pad_id)
  labels = torch.clamp(labels, min=0)
  nll_loss = log_probs.gather(dim=-1, index=labels)
  smoothed_loss = log_probs.sum(dim=-1, keepdim=True, dtype=torch.bfloat16)
  nll_loss.masked_fill_(padding_mask, 0.0)
  smoothed_loss.masked_fill_(padding_mask, 0.0)
  num_active_elements = padding_mask.numel() - padding_mask.long().sum()
  nll_loss = nll_loss.sum() / num_active_elements
  smoothed_loss = smoothed_loss.sum() / (num_active_elements * log_probs.shape[-1])
  del labels,logits,padding_mask
  return (1-epsilon)*nll_loss + epsilon*smoothed_loss



def train(FLAGS):


    ### Configuring Training
    global val_step
    update_params= filter(lambda p: p.requires_grad, model.parameters())
    num_iterations = int((FLAGS["NUM_EPOCHS"] * FLAGS['STEPS'] ) // FLAGS['GRAD_ACCUMULATION_STEP'])
    warmup_steps = int(num_iterations * FLAGS['WARMUP_RATIO'])

    if __wandb__:
        wandb.init(project=FLAGS['PROJECT'],config=FLAGS)
        wandb.define_metric("Validation_loss", step_metric="val_step")
        wandb.define_metric("Learning_rate",step_metric="train_step")
        wandb.define_metric("train_loss",step_metric="train_step")

    ### Optimizers

    if (FLAGS['OPTIMIZER']).lower()=='adamw':
        optimizer = AdamW(update_params, eps=1e-8, lr=FLAGS['LEARNING_RATE'], betas=(0.9, 0.999),weight_decay=FLAGS['WEIGHT_DECAY'])
    elif (FLAGS['OPTIMIZER']).lower()=='lion':
        optimizer = Lion(update_params, lr=FLAGS['LEARNING_RATE'],weight_decay=FLAGS['WEIGHT_DECAY'])
    elif (FLAGS['OPTIMIZER']).lower()=='adafactor':
        optimizer = Adafactor(update_params,lr=FLAGS['LEARNING_RATE'],weight_decay=FLAGS['WEIGHT_DECAY'],scale_parameter=False,relative_step=False)
    elif (FLAGS['OPTIMIZER']).lower()=='came':
        optimizer = CAME(model.parameters(),lr=FLAGS['LEARNING_RATE'],weight_decay=FLAGS['WEIGHT_DECAY'],betas=(0.9, 0.999, 0.9999),eps=(1e-30, 1e-16))
    else:
#         optimizer = Lilith(update_params, eps=1e-8, lr=FLAGS['LEARNING_RATE'],weight_decay=FLAGS['WEIGHT_DECAY'])
        optimizer=SM3(update_params,lr=FLAGS['LEARNING_RATE'])

    for param_group in optimizer.param_groups:
        if len(param_group["params"]) > 0:
            print(param_group["params"][0].device)
            break


    ### Schedulars

    if (FLAGS['SCHEDULAR']).lower()=='linear':
        scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps,num_iterations)
    else:
        scheduler = get_cosine_schedule_with_warmup(optimizer,warmup_steps,num_iterations)




    ### Training Loop
    val_step=0
    check=False #for brakes
    for epoch in range(1, FLAGS['NUM_EPOCHS'] + 1):
        if check:
            break
        model.train()
        xm.master_print('Epoch {} train begin {}'.format(epoch, test_utils.now()))
        for step, batch in enumerate(training_loader):
            input_ids, labels,attention_mask = batch["input_ids"].to(device),  batch["labels"].to(device),batch['attention_mask'].to(device)
            xs.mark_sharding(input_ids, mesh, (0,1))  ### earlier:-> (0,1) according to pytorch-xla , input/dataloaders must be sharded across ('data',None)
            xs.mark_sharding( labels,  mesh, (0,1))  ###
            xs.mark_sharding(  attention_mask,  mesh, (0, 1)) ###
            outputs = model(input_ids=input_ids,attention_mask=attention_mask)
            loss = evaluate_loss(outputs,labels)


            if (step + 1) % FLAGS['LOGGING_STEPS'] == 0:
                xm.master_print(f'loss: {loss.detach().cpu().item()}, time: {test_utils.now()}, step: {step+1}')
            if __wandb__:
                wandb.log({
                'Learning_rate': optimizer.param_groups[0]['lr'],
                'train_loss':  loss.detach().cpu().item(),
                'train_step': step + 1 + ((epoch-1) * FLAGS["STEPS"]),
                        })




            del input_ids , attention_mask
            loss.backward()
            del outputs,loss




            if (step+1) % FLAGS['GRAD_ACCUMULATION_STEP'] == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=FLAGS['MAX_GRAD_CLIP']*8)
                scheduler.step()
                xm.optimizer_step(optimizer,pin_layout=True,barrier=True) ## performs xm.reduce_gradient() , optimizer.step() , xm.mark_step()
                optimizer.zero_grad()





            if (step+1)% FLAGS['VAL_STEPS'] == 0:
                end_index=FLAGS["VAL_BATCH"]
                model.eval()
                with torch.no_grad():
                    total_loss = 0
                    total_step = 0
                    for stepx, batchx in enumerate(testing_loader):
                        input_ids = batchx["input_ids"].to(device)
                        labels = batchx["labels"].to(device)
                        attention_mask = batchx["attention_mask"].to(device)
                        xs.mark_sharding(input_ids, mesh, (0, None))
                        xs.mark_sharding(labels, mesh, (0, None))
                        xs.mark_sharding( attention_mask,    mesh, (0, None))
                        outputs = model(input_ids=input_ids,attention_mask=attention_mask)
                        loss = evaluate_loss(outputs,labels)
                        total_loss += loss.item()
                        total_step +=1
                        xm.master_print('----- Time -> {} ----- Validation Batch -> {} ----  Validation Loss -> {:.4f}'.format(test_utils.now(), total_step , loss.item()))
                        if __wandb__:
                            val_step+=1
                            wandb.log({
                                'Validation_loss': loss.item(),
                                'val_step':val_step,
                                    })
                        if (stepx+1)%end_index==0:
                            break
                    model.train()
                    average_loss=total_loss/total_step
                    xm.master_print('----- Time -> {} ----- Validation Batch Size -> {} ----  Validation Loss -> {:.7f}'.format(test_utils.now(), total_step , average_loss))

            if (step+1)% FLAGS['PAUSE_STEPS']==0:
                inp=input('want to continue training after {} steps'.format(step+1))
                check = bool("no" in inp.lower())
                if check:
                    break
                else:
                    pass

In [ ]:
train(FLAGS)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


xla:0
Epoch 1 train begin 19:07:00


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


loss: 2.0557291507720947, time: 19:07:45, step: 1
loss: 2.1863489151000977, time: 19:09:15, step: 2
loss: 2.0923361778259277, time: 19:10:58, step: 3
loss: 2.152149200439453, time: 19:11:04, step: 4
loss: 2.03597092628479, time: 19:11:09, step: 5
loss: 2.006934404373169, time: 19:11:15, step: 6
loss: 1.9849658012390137, time: 19:11:20, step: 7
loss: 2.0092203617095947, time: 19:11:26, step: 8
loss: 2.1371185779571533, time: 19:11:31, step: 9
loss: 1.9907788038253784, time: 19:11:37, step: 10
loss: 1.9026447534561157, time: 19:11:42, step: 11
loss: 1.9469952583312988, time: 19:11:48, step: 12
loss: 1.917063593864441, time: 19:11:53, step: 13
loss: 1.8778440952301025, time: 19:11:58, step: 14
loss: 2.0275919437408447, time: 19:12:04, step: 15
loss: 2.18735933303833, time: 19:12:09, step: 16
loss: 2.0178208351135254, time: 19:12:15, step: 17
loss: 1.9779554605484009, time: 19:12:20, step: 18
loss: 2.1814520359039307, time: 19:12:26, step: 19
loss: 1.79534912109375, time: 19:12:31, step: 2

want to continue training after 1000 steps 


loss: 1.7331489324569702, time: 20:46:39, step: 1001
loss: 1.7081866264343262, time: 20:46:45, step: 1002
loss: 1.8107364177703857, time: 20:46:50, step: 1003
loss: 1.8283324241638184, time: 20:46:55, step: 1004
loss: 1.685908555984497, time: 20:47:01, step: 1005
loss: 1.8649673461914062, time: 20:47:06, step: 1006
loss: 1.5657192468643188, time: 20:47:12, step: 1007
loss: 1.7067697048187256, time: 20:47:17, step: 1008
loss: 1.9105116128921509, time: 20:47:23, step: 1009
loss: 1.7716692686080933, time: 20:47:28, step: 1010
loss: 1.6767313480377197, time: 20:47:34, step: 1011
loss: 1.8783650398254395, time: 20:47:39, step: 1012
loss: 1.8751152753829956, time: 20:47:44, step: 1013
loss: 1.723248839378357, time: 20:47:50, step: 1014
loss: 1.8573319911956787, time: 20:47:55, step: 1015
loss: 1.636582374572754, time: 20:48:01, step: 1016
loss: 1.7426223754882812, time: 20:48:06, step: 1017
loss: 1.5691308975219727, time: 20:48:12, step: 1018
loss: 1.9785736799240112, time: 20:48:17, step: 1

want to continue training after 2000 steps no


## Saving the Model Trained for 2000 Steps on HuggingFace

In [ ]:
import time
print('Loading the model on CPU')
START=time.time()
model = model.cpu()
print(f"Loaded model on cpu in {time.time()-START} seconds ")

Loading the model on CPU
Loaded model on cpu in 113.14982438087463 seconds 


In [ ]:
from huggingface_hub import login
login(hf_token) ##
model.push_to_hub(
    SAVED_MODEL,
    tokenizer=tokenizer,
    safe_serialization=True,
    create_pr=True,
    max_shard_size="3GB",
)

tokenizer.push_to_hub(
    SAVED_MODEL,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


adapter_model.safetensors: 100%|██████████| 503M/503M [00:16<00:00, 30.4MB/s] 
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Alexis-Az/Story-Generation-LlaMA-8B/commit/96fe00d4c6b770c0fef83f9921e652ce34a0144a', commit_message='Upload tokenizer', commit_description='', oid='96fe00d4c6b770c0fef83f9921e652ce34a0144a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Alexis-Az/Story-Generation-LlaMA-8B', endpoint='https://huggingface.co', repo_type='model', repo_id='Alexis-Az/Story-Generation-LlaMA-8B'), pr_revision=None, pr_num=None)